# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print('The current working directory is: \n {}\n'.format(os.getcwd()))

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print('The file path list has {} paths inside.'.format(len(file_path_list)))

The current working directory is: 
 /Users/djanmagno/Documents/Data_Science/Udacity-Data-Engineering-Nanodegree/Project-2-Data-Modeling-with-Apache-Cassandra

The file path list has 30 paths inside.


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Total number of rows: {}\n'.format(len(full_data_rows_list)))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

try:
    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

    print('CSV file created with success.')
except Exception as e:
    # Printing error
    print('The error was: {}'.format(e))

Total number of rows: 8056

CSV file created with success.


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('The number of rows in the CSV file is: {}'.format(sum(1 for line in f)))

The number of rows in the CSV file is: 6821


# Part II. Apache Cassandra coding portion of the project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster

try:
    # Making a connection to a Cassandra instance my local machine
    cluster = Cluster([('127.0.0.1')])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print('Connection with Cassandra established with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

Connection with Cassandra established with success.


#### Create Keyspace

In [6]:
# Creating the Keyspace 
query = """
CREATE KEYSPACE IF NOT EXISTS sparkify_ks
WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
"""
try:
    # Creating keyspace sparkify_ks
    session.execute(query)
    print('KEYSPACE sparkify_ks created with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)    

KEYSPACE sparkify_ks created with success.


#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    # Setting KEYSPACE to sparkify_ks
    session.set_keyspace('sparkify_ks')
    print('KEYSPACE set to sparkify_ks with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

KEYSPACE set to sparkify_ks with success.


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

```
    SELECT artist, song, lenght FROM event_history WHERE sessionId = 338 AND itemInSession  = 4
```

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```
    SELECT artist, song, firstName, lastName FROM event_history WHERE userid = 10 AND sessionid = 182       
```

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```
    SELECT firstName, LastName FROM event_history WHERE song = 'All Hands Against His Own'
```

# Data Modeling with Apache Cassandra

## Query 1 : Return the artist, song title and song's length in the music app history that was heard during a given sessionId and itemInSession

In order to execute the Query above in Apache Cassandra, first we need to create a table modeled to this Query.

For this, we will have as PRIMARY KEY the attribute `sessionId` as **Partition Key** and `itemInSession` as ours **Clustering Key**. The `sessionId` will be **hashed** to be divided among the Nodes and the query will use `itemInSession` to sort the `sessionId` in **Ascending** order.

### Creating the Table `artist_library`

In [8]:
# Creating the table artist_library for the First Query
query = "CREATE TABLE IF NOT EXISTS artist_library "

query += "(session_id int, item_in_session int, artist_name varchar, song_title text, \
    song_length decimal, PRIMARY KEY((session_id), item_in_session))"

try:
    # Creating the table for query 01
    session.execute(query)
    print('Table artist_library created with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)                 

Table artist_library created with success.


### Inserting data into the Table `artist_library`

In [9]:
file = 'event_datafile_new.csv'

# Opening event_datafile_new.csv
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header

    # Loop to pass in each line of the csv file
    for line in csvreader:
        # Assigning the INSERT statement into the `query` variable
        query = """
            INSERT INTO artist_library 
                (
                    session_id, 
                    item_in_session, 
                    artist_name, 
                    song_title, 
                    song_length
                )
        """
        query += " VALUES (%s, %s, %s, %s, %s)"
        
        # Assigning the right values for each column.
        try:
            # Inserting values on table artist_library
            session.execute(
                query, 
                (
                    int(line[8]), 
                    int(line[3]), 
                    line[0], 
                    line[9], 
                    float(line[5])
                )
            )

        except Exception as e:
            # Printing EXCEPTION in case of error
            print(e)

print('Values inserted on table artist_library.')

Values inserted on table artist_library.


### Running the Query 01

In [10]:
# Query 01
query = """
SELECT 
    artist_name, 
    song_title, 
    song_length 
FROM 
    artist_library 
WHERE 
    session_id = 338 
        AND 
    item_in_session  = 4 
"""

try:
    # Executing the Query
    rows = session.execute(query)
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

print('The return for Query 01 is:\n')
for row in rows:
    # Printing the return of the Query 01
    print(row.artist_name, row.song_title, row.song_length)

The return for Query 01 is:

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 02 : Return only the following requirements: name of artist, song (sorted by itemInSession) and user (first and last name) for a specific userid and sessionid

In order to execute the Query above in Apache Cassandra, it is necessary to create a table modeled specifically to this Query.

For this, we will have as PRIMARY KEY the attributes ((`userId`, `sessionId`), `itemInSession`). The **Partition Key** will be represented by (`userId`, `sessionId`) because if we choose to use only `userId` as **Partition Key** it is very likely that there will not have a fairl distribution across the node since there are some repeated `userId`. 

That is the reason why we should use both (`userId`, `sessionId`) as **Partition Key** to avoid having a concentration of data in one node for instance.

The `itemInSession` will be used as our **Clustering Key**. The `userId` and `sessionId` will be **hashed** to be divided among the Nodes and the query will use `itemInSession` to sort the data a in **Ascending** order.

### Creating Table `user_library`

In [11]:
query = "CREATE TABLE IF NOT EXISTS user_library "

query += "(user_id int, session_id int, item_in_session int, first_name varchar, last_name varchar, \
    artist_name varchar, song_title text, PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    # Creating table user_library
    session.execute(query)
    print('Table user_library created with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)        

Table user_library created with success.


### Inserting data into Table `user_library`

In [12]:
# Opening event_datafile_new.csv
with open(file, encoding = 'utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader)

    for line in csvreader:
        # Assigning the INSERT statement into the `query` variable
        query = """
            INSERT INTO user_library
                (
                    user_id,
                    session_id,
                    item_in_session,
                    first_name,
                    last_name,
                    artist_name,
                    song_title
                )
        """ 

        query += " VALUES (%s, %s, %s, %s, %s, %s, %s)" 

        # Assigning the right values for each column on table user_library
        try:
            # Inserting values on table user_library
            session.execute(
                query,
                (
                    int(line[10]),
                    int(line[8]),
                    int(line[3]),
                    line[1],
                    line[4],
                    line[0],
                    line[9]
                )
            )
        except Exception as e:
            # Printing EXCEPTION in case of error
            print(e)
print('Values inserted on table user_library.')

Values inserted on table user_library.


### Running Query 02

In [13]:
# Query 02
query = """
SELECT
    artist_name,
    song_title,
    first_name,
    last_name
FROM 
    user_library
WHERE
    user_id = 10
        AND
    session_id = 182
"""

try:
    # Executing the query
    rows = session.execute(query)
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

print('The return for Query 02 is:\n')
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name)

The return for Query 02 is:

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 03 : Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In order to execute the Query above in Apache Cassandra, it is necessary to create a table modeled specifically to this Query.

For this reason, we will have as PRIMARY KEY the attributes ((`song`), `userId`). The **Partition Key** will be represented by `song` since it is what we want to query and our **Clustering Key** will be represented by `userId` which will hash our **Partition Key**, give the uniquiness to the PRIMARY KEY, and sort the data a in a **Ascending** order.

### Create Table `user_history`

In [14]:
query = "CREATE TABLE IF NOT EXISTS users_history "

query += "(song_title text, user_id int, first_name varchar, last_name varchar, \
    PRIMARY KEY(song_title, user_id))"

try:
    # Creating table user_history
    session.execute(query)
    print('Table users_history created with success.')
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

Table users_history created with success.


### Inserting data into Table `user_history`

In [15]:
with open(file, encoding = 'utf-8') as f:
    csvreader = csv.reader(f)
    next(csvreader)

    for line in csvreader:
    ## Assigning the INSERT statement into the `query` variable
        query = """
            INSERT INTO users_history
                (
                    song_title,
                    user_id,
                    first_name,
                    last_name     
                )
        """ 

        query += " VALUES (%s, %s, %s, %s)" 

        # Assigning the right values for each column on table users_history
        try:
            # Inserting values on table user_library
            session.execute(
                query,
                (
                    line[9],
                    int(line[10]),
                    line[1],
                    line[4]
                )
            )
        except Exception as e:
            # Printing EXCEPTION in case of error
            print(e)
print('Values inserted on table users_history.')

Values inserted on table users_history.


### Running Query 03

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
SELECT
    first_name,
    last_name
FROM
    users_history
WHERE
    song_title = 'All Hands Against His Own'
"""

try:
    # Executing the query
    rows = session.execute(query)
except Exception as e:
    # Printing EXCEPTION in case of error
    print(e)

print('The return for Query 03 is:\n')
for row in rows:
    print(row.first_name, row.last_name)                  

The return for Query 03 is:

Jacqueline Lynch
Tegan Levine
Sara Johnson


## Drop the tables before closing out the sessions

In [17]:
# Dropping the tables 
table_list = ['artist_library', 'user_library', 'users_history']

for table in table_list:
    query = "DROP TABLE {}".format(table)
    try:
        # Executing the query
        session.execute(query)
        print('Table {} dropped.\n'.format(table))
    except Exception as e:
        # Printing EXCEPTION in case of error
        print(e)

Table artist_library dropped.

Table user_library dropped.

Table users_history dropped.



### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()